In [1]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random
from collections import deque
from sys import maxsize
import pickle

In [2]:
def createList(r1, r2):#option creation for random edge addition
    if (r1 == r2):
        return r1
    else:
        res = []
        while(r1 != r2):
            res.append(r1)
            r1 += 1
            if (r1 > 50):
                r1 = r1-50
        res.append(r2)
    return res 

In [3]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
        self.paths = self.all_shortest_paths()
        
    def find_paths(self, paths, path, parent, end):
        if end == -1:
            paths.append(path.copy())
            return

        for par in parent[end]:
            path.append(end)
            self.find_paths(paths, path, parent, par)
            path.pop()
        
    def bfs_all_paths(self, parent, start):
    
        dist = [maxsize for _ in range(51)]
        q = deque()
        q.append(start)
        parent[start] = [-1]
        dist[start] = 0

        while q:
            u = q[0]
            q.popleft()
            for v in self.graph[u]:
                if dist[v] > dist[u] + 1:
                    dist[v] = dist[u] + 1
                    q.append(v)
                    parent[v].clear()
                    parent[v].append(u)

                elif dist[v] == dist[u] + 1:
                    parent[v].append(u)

    def all_shortest_paths(self):
        shortest_paths = dict()
        for i in range(1, 51):
            for j in range(1, 51):
                paths, path, shortest_path = [], [], []

                parent = [[] for _ in range(51)]
                self.bfs_all_paths(parent, i)

                self.find_paths(paths, path, parent, j)

                for move in paths:
                    move = list(reversed(move))
                    shortest_path.append(move)
                    
                shortest_paths[i,j] = shortest_path

        return shortest_paths

                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1): #deleting adjacent options so that double edges arent made
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else: 
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [4]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [5]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
    
    def pred_distracted_move(self, g, agentpos): #predator moving with probability 0.6 to node closer to agent and probability 0.4 to any other random neighbour
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths 
        new_path = random.randint(0,(len(shortest_path)-1))
        chance = random.randint(1,100)
        if chance <= 60: #60 percent chance of moving correctly
            self.pos = shortest_path[new_path][0]
        else: #40 percent chance of moving in distracted manner
            self.pos = random.choice(neighbors)
        return self
        

In [6]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [7]:
#running a neural network that estimates the UStar values
class VStar:
    
    #initializing neural network with features W0 and W1
    def __init__(self,x,y):
        self.W0 = np.random.randn(36, 6)*0.01
        self.W1 = np.random.randn(1, 36)*0.01
        self.h0 = np.zeros((36, 1))
        self.h1 = np.zeros((1, 1))
        self.loss_array = []
        self.sigma0 = None
        self.sigma1 = None
        self.alpha0 = None
        self.alpha1 = None
        self.x = x.T
        self.y = y.T
                
    #Leaky linear unit activation function
    def leak_lu(self,arr):
        arr[arr<0] = arr[arr<0]*0.01
        return arr
    
    #exponential linear unit activation function
    def exp_lu(self,arr):
        arr[arr<0] = 0.01
        arr[arr>=0] = 1
        return arr
     
    #run through the data forwards
    def forward(self):
        self.sigma0 = np.dot(self.W0,self.x) + self.h0
        self.alpha0 = self.leak_lu(self.sigma0)
        self.sigma1 = np.dot(self.W1,self.alpha0) + self.h1
        self.alpha1 = self.sigma1
    
    #run through the data backwards
    def backward(self):
        back_sigma1 = (self.alpha1-self.y)
        back_W1 = (1/self.x.shape[1])*(np.dot(back_sigma1,self.alpha0.T))
        back_h1 = (1/self.x.shape[1]) * np.sum(back_sigma1)
        back_sigma0 = np.multiply(np.dot(self.W1.T,back_sigma1), self.exp_lu(self.sigma1))
        back_W0 = (1/self.x.shape[1])*(np.dot(back_sigma0,self.x.T))
        back_h0 = (1/self.x.shape[1])*(np.sum(back_sigma0))
        return back_W0, back_h0, back_W1, back_h1
    
    #getting the values forward and running backwards to get the final estimations and loss
    def run(self):
        #iterating to reduce loss
        for i in range(3000):
            self.forward()
            back_W0, back_h0, back_W1, back_h1 = self.backward()
            self.W0 = self.W0 - 0.001*back_W0
            self.h0 = self.h0 - 0.001*back_h0
            self.W1 = self.W1 - 0.001*back_W1
            self.h1 = self.h1 - 0.001*back_h1
            loss = (np.mean((self.y - self.alpha1)**2)*0.5)
            self.loss_array.append(loss)
            print("Loss at: ", i,loss)
    
    # using the values from the neural network to determine the UStar(s) value of x
    def predict(self, x):
        alpha0 = np.maximum(np.dot(self.W0,x) + self.h0, 0)
        sigma1 = np.dot(self.W1,alpha0) + self.h1
        return sigma1

In [8]:
#using a specific graph and UStar map to train the data
f = open('ustar-dion.p', 'rb')
utility = pickle.load(f)
f.close()
g = Graph()
g.graph = {1: [2, 50, 5], 2: [3, 1, 50], 3: [4, 2, 7], 4: [5, 3, 9], 5: [6, 4, 1], 6: [7, 5, 8], 7: [8, 6, 3], 8: [9, 7, 6], 9: [10, 8, 4], 10: [11, 9, 14], 11: [12, 10], 12: [13, 11, 16], 13: [14, 12, 18], 14: [15, 13, 10], 15: [16, 14, 19], 16: [17, 15, 12], 17: [18, 16, 22], 18: [19, 17, 13], 19: [20, 18, 15], 20: [21, 19, 24], 21: [22, 20, 26], 22: [23, 21, 17], 23: [24, 22, 27], 24: [25, 23, 20], 25: [26, 24, 29], 26: [27, 25, 21], 27: [28, 26, 23], 28: [29, 27, 30], 29: [30, 28, 25], 30: [31, 29, 28], 31: [32, 30, 34], 32: [33, 31], 33: [34, 32, 36], 34: [35, 33, 31], 35: [36, 34, 39], 36: [37, 35, 33], 37: [38, 36, 40], 38: [39, 37, 41], 39: [40, 38, 35], 40: [41, 39, 37], 41: [42, 40, 38], 42: [43, 41, 45], 43: [44, 42, 48], 44: [45, 43, 46], 45: [46, 44, 42], 46: [47, 45, 44], 47: [48, 46, 49], 48: [49, 47, 43], 49: [50, 48, 47], 50: [1, 49, 2]}
g.paths = g.all_shortest_paths()

In [9]:
#saving all the positions, distances, and ustar values in a dataframe for each state for training and testing the neural network
util = []
for pos,val in utility.items():
    local = {}
    local["Agent"] = pos[0]
    local["Prey"] = pos[1]
    local["Predator"] = pos[2]
    local["Distance between Agent and Prey"] = len(g.paths.get((pos[0],pos[1]))[0])
    local["Distance between Agent and Predator"] = len(g.paths.get((pos[0],pos[2]))[0])
    local["Distance between Prey and Predator"] = len(g.paths.get((pos[2],pos[1]))[0])
    local["Ustar"] = val
    util.append(local)

#total = dataframe for training and testing
total = pd.DataFrame(util)
total_copy = total.copy()
total = total[total['Ustar'] < 100]

#x = agent, prey, and predator positions and distances between them
x = total.iloc[:,0:6].values
#y = UStar values
y = np.expand_dims(total.iloc[:,6].values, axis = 1)

#creating the training and testing data
train1, train2, test1, test2 = [], [], [], []

#70% of the total dataframe are for training, 30% are for testing
for i in range(len(y)):
    prob = np.random.randn()
    if prob < 0.7:
        train1.append(total.iloc[i,:-1].values)
        train2.append(total.iloc[i,-1])
    else:
        test1.append(total.iloc[i,:-1].values)
        test2.append(total.iloc[i,-1])

train1 = np.array(train1)
train2 = np.expand_dims(np.array(train2), axis = 1)
test1 = np.array(test1)
test2 = np.expand_dims(test2, axis = 1)


In [10]:
network1 = VStar(x,y)
network1.run()

Loss at:  0 31.123627849755543
Loss at:  1 31.123627849755543
Loss at:  2 31.123627849755543
Loss at:  3 31.123627849755543
Loss at:  4 31.123627849755543
Loss at:  5 31.123627849755543
Loss at:  6 31.123627849755543
Loss at:  7 31.123627849755543
Loss at:  8 31.123627849755543
Loss at:  9 31.123627849755543
Loss at:  10 31.123627849755543
Loss at:  11 31.123627849755543
Loss at:  12 31.123627849755543
Loss at:  13 31.123627849755543
Loss at:  14 31.123627849755543
Loss at:  15 31.123627849755543
Loss at:  16 31.123627849755543
Loss at:  17 31.123627849755543
Loss at:  18 31.123627849755543
Loss at:  19 31.123627849755543
Loss at:  20 31.123627849755543
Loss at:  21 31.123627849755543
Loss at:  22 31.123627849755543
Loss at:  23 31.123627849755543
Loss at:  24 31.123627849755543
Loss at:  25 31.123627849755543
Loss at:  26 31.123627849755543
Loss at:  27 31.123627849755543
Loss at:  28 31.123627849755543
Loss at:  29 31.123627849755543
Loss at:  30 31.123627849755543
Loss at:  31 31.12

Loss at:  253 31.123627849755543
Loss at:  254 31.123627849755543
Loss at:  255 31.123627849755543
Loss at:  256 31.123627849755543
Loss at:  257 31.123627849755543
Loss at:  258 31.123627849755543
Loss at:  259 31.123627849755543
Loss at:  260 31.123627849755543
Loss at:  261 31.123627849755543
Loss at:  262 31.123627849755543
Loss at:  263 31.123627849755543
Loss at:  264 31.123627849755543
Loss at:  265 31.123627849755543
Loss at:  266 31.123627849755543
Loss at:  267 31.123627849755543
Loss at:  268 31.123627849755543
Loss at:  269 31.123627849755543
Loss at:  270 31.123627849755543
Loss at:  271 31.123627849755543
Loss at:  272 31.123627849755543
Loss at:  273 31.123627849755543
Loss at:  274 31.123627849755543
Loss at:  275 31.123627849755543
Loss at:  276 31.123627849755543
Loss at:  277 31.123627849755543
Loss at:  278 31.123627849755543
Loss at:  279 31.123627849755543
Loss at:  280 31.123627849755543
Loss at:  281 31.123627849755543
Loss at:  282 31.123627849755543
Loss at:  

Loss at:  505 31.123627849755543
Loss at:  506 31.123627849755543
Loss at:  507 31.123627849755543
Loss at:  508 31.123627849755543
Loss at:  509 31.123627849755543
Loss at:  510 31.123627849755543
Loss at:  511 31.123627849755543
Loss at:  512 31.123627849755543
Loss at:  513 31.123627849755543
Loss at:  514 31.123627849755543
Loss at:  515 31.123627849755543
Loss at:  516 31.123627849755543
Loss at:  517 31.123627849755543
Loss at:  518 31.123627849755543
Loss at:  519 31.123627849755543
Loss at:  520 31.123627849755543
Loss at:  521 31.123627849755543
Loss at:  522 31.123627849755543
Loss at:  523 31.123627849755543
Loss at:  524 31.123627849755543
Loss at:  525 31.123627849755543
Loss at:  526 31.123627849755543
Loss at:  527 31.123627849755543
Loss at:  528 31.123627849755543
Loss at:  529 31.123627849755543
Loss at:  530 31.123627849755543
Loss at:  531 31.123627849755543
Loss at:  532 31.123627849755543
Loss at:  533 31.123627849755543
Loss at:  534 31.123627849755543
Loss at:  

Loss at:  756 31.123627849755543
Loss at:  757 31.123627849755543
Loss at:  758 31.123627849755543
Loss at:  759 31.123627849755543
Loss at:  760 31.123627849755543
Loss at:  761 31.123627849755543
Loss at:  762 31.123627849755543
Loss at:  763 31.123627849755543
Loss at:  764 31.123627849755543
Loss at:  765 31.123627849755543
Loss at:  766 31.123627849755543
Loss at:  767 31.123627849755543
Loss at:  768 31.123627849755543
Loss at:  769 31.123627849755543
Loss at:  770 31.123627849755543
Loss at:  771 31.123627849755543
Loss at:  772 31.123627849755543
Loss at:  773 31.123627849755543
Loss at:  774 31.123627849755543
Loss at:  775 31.123627849755543
Loss at:  776 31.123627849755543
Loss at:  777 31.123627849755543
Loss at:  778 31.123627849755543
Loss at:  779 31.123627849755543
Loss at:  780 31.123627849755543
Loss at:  781 31.123627849755543
Loss at:  782 31.123627849755543
Loss at:  783 31.123627849755543
Loss at:  784 31.123627849755543
Loss at:  785 31.123627849755543
Loss at:  

Loss at:  1008 31.123627849755543
Loss at:  1009 31.123627849755543
Loss at:  1010 31.123627849755543
Loss at:  1011 31.123627849755543
Loss at:  1012 31.123627849755543
Loss at:  1013 31.123627849755543
Loss at:  1014 31.123627849755543
Loss at:  1015 31.123627849755543
Loss at:  1016 31.123627849755543
Loss at:  1017 31.123627849755543
Loss at:  1018 31.123627849755543
Loss at:  1019 31.123627849755543
Loss at:  1020 31.123627849755543
Loss at:  1021 31.123627849755543
Loss at:  1022 31.123627849755543
Loss at:  1023 31.123627849755543
Loss at:  1024 31.123627849755543
Loss at:  1025 31.123627849755543
Loss at:  1026 31.123627849755543
Loss at:  1027 31.123627849755543
Loss at:  1028 31.123627849755543
Loss at:  1029 31.123627849755543
Loss at:  1030 31.123627849755543
Loss at:  1031 31.123627849755543
Loss at:  1032 31.123627849755543
Loss at:  1033 31.123627849755543
Loss at:  1034 31.123627849755543
Loss at:  1035 31.123627849755543
Loss at:  1036 31.123627849755543
Loss at:  1037

...

Loss at:  1981 31.123627849755543
Loss at:  1982 31.123627849755543
Loss at:  1983 31.123627849755543
Loss at:  1984 31.123627849755543
Loss at:  1985 31.123627849755543
Loss at:  1986 31.123627849755543
Loss at:  1987 31.123627849755543
Loss at:  1988 31.123627849755543
Loss at:  1989 31.123627849755543
Loss at:  1990 31.123627849755543
Loss at:  1991 31.123627849755543
Loss at:  1992 31.123627849755543
Loss at:  1993 31.123627849755543
Loss at:  1994 31.123627849755543
Loss at:  1995 31.123627849755543
Loss at:  1996 31.123627849755543
Loss at:  1997 31.123627849755543
Loss at:  1998 31.123627849755543
Loss at:  1999 31.123627849755543
Loss at:  2000 31.123627849755543
Loss at:  2001 31.123627849755543
Loss at:  2002 31.123627849755543
Loss at:  2003 31.123627849755543
Loss at:  2004 31.123627849755543
Loss at:  2005 31.123627849755543
Loss at:  2006 31.123627849755543
Loss at:  2007 31.123627849755543
Loss at:  2008 31.123627849755543
Loss at:  2009 31.123627849755543
Loss at:  2010

Loss at:  2223 31.123627849755543
Loss at:  2224 31.123627849755543
Loss at:  2225 31.123627849755543
Loss at:  2226 31.123627849755543
Loss at:  2227 31.123627849755543
Loss at:  2228 31.123627849755543
Loss at:  2229 31.123627849755543
Loss at:  2230 31.123627849755543
Loss at:  2231 31.123627849755543
Loss at:  2232 31.123627849755543
Loss at:  2233 31.123627849755543
Loss at:  2234 31.123627849755543
Loss at:  2235 31.123627849755543
Loss at:  2236 31.123627849755543
Loss at:  2237 31.123627849755543
Loss at:  2238 31.123627849755543
Loss at:  2239 31.123627849755543
Loss at:  2240 31.123627849755543
Loss at:  2241 31.123627849755543
Loss at:  2242 31.123627849755543
Loss at:  2243 31.123627849755543
Loss at:  2244 31.123627849755543
Loss at:  2245 31.123627849755543
Loss at:  2246 31.123627849755543
Loss at:  2247 31.123627849755543
Loss at:  2248 31.123627849755543
Loss at:  2249 31.123627849755543
Loss at:  2250 31.123627849755543
Loss at:  2251 31.123627849755543
Loss at:  2252

Loss at:  2467 31.123627849755543
Loss at:  2468 31.123627849755543
Loss at:  2469 31.123627849755543
Loss at:  2470 31.123627849755543
Loss at:  2471 31.123627849755543
Loss at:  2472 31.123627849755543
Loss at:  2473 31.123627849755543
Loss at:  2474 31.123627849755543
Loss at:  2475 31.123627849755543
Loss at:  2476 31.123627849755543
Loss at:  2477 31.123627849755543
Loss at:  2478 31.123627849755543
Loss at:  2479 31.123627849755543
Loss at:  2480 31.123627849755543
Loss at:  2481 31.123627849755543
Loss at:  2482 31.123627849755543
Loss at:  2483 31.123627849755543
Loss at:  2484 31.123627849755543
Loss at:  2485 31.123627849755543
Loss at:  2486 31.123627849755543
Loss at:  2487 31.123627849755543
Loss at:  2488 31.123627849755543
Loss at:  2489 31.123627849755543
Loss at:  2490 31.123627849755543
Loss at:  2491 31.123627849755543
Loss at:  2492 31.123627849755543
Loss at:  2493 31.123627849755543
Loss at:  2494 31.123627849755543
Loss at:  2495 31.123627849755543
Loss at:  2496

Loss at:  2709 31.123627849755543
Loss at:  2710 31.123627849755543
Loss at:  2711 31.123627849755543
Loss at:  2712 31.123627849755543
Loss at:  2713 31.123627849755543
Loss at:  2714 31.123627849755543
Loss at:  2715 31.123627849755543
Loss at:  2716 31.123627849755543
Loss at:  2717 31.123627849755543
Loss at:  2718 31.123627849755543
Loss at:  2719 31.123627849755543
Loss at:  2720 31.123627849755543
Loss at:  2721 31.123627849755543
Loss at:  2722 31.123627849755543
Loss at:  2723 31.123627849755543
Loss at:  2724 31.123627849755543
Loss at:  2725 31.123627849755543
Loss at:  2726 31.123627849755543
Loss at:  2727 31.123627849755543
Loss at:  2728 31.123627849755543
Loss at:  2729 31.123627849755543
Loss at:  2730 31.123627849755543
Loss at:  2731 31.123627849755543
Loss at:  2732 31.123627849755543
Loss at:  2733 31.123627849755543
Loss at:  2734 31.123627849755543
Loss at:  2735 31.123627849755543
Loss at:  2736 31.123627849755543
Loss at:  2737 31.123627849755543
Loss at:  2738

Loss at:  2953 31.123627849755543
Loss at:  2954 31.123627849755543
Loss at:  2955 31.123627849755543
Loss at:  2956 31.123627849755543
Loss at:  2957 31.123627849755543
Loss at:  2958 31.123627849755543
Loss at:  2959 31.123627849755543
Loss at:  2960 31.123627849755543
Loss at:  2961 31.123627849755543
Loss at:  2962 31.123627849755543
Loss at:  2963 31.123627849755543
Loss at:  2964 31.123627849755543
Loss at:  2965 31.123627849755543
Loss at:  2966 31.123627849755543
Loss at:  2967 31.123627849755543
Loss at:  2968 31.123627849755543
Loss at:  2969 31.123627849755543
Loss at:  2970 31.123627849755543
Loss at:  2971 31.123627849755543
Loss at:  2972 31.123627849755543
Loss at:  2973 31.123627849755543
Loss at:  2974 31.123627849755543
Loss at:  2975 31.123627849755543
Loss at:  2976 31.123627849755543
Loss at:  2977 31.123627849755543
Loss at:  2978 31.123627849755543
Loss at:  2979 31.123627849755543
Loss at:  2980 31.123627849755543
Loss at:  2981 31.123627849755543
Loss at:  2982

In [11]:
def AgentVStar(g, utility):
    agent_pos = random.randint(1,50)
    predator = Predator(agent_pos)
    prey = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    
    while (fail == 0 and steps <= 100):
        
        #move agent to the neighbor with the lowest utility
        best_node = agent_pos
        min_utility = 10000
        
        for action in g.graph[agent_pos]:
            if utility[action][prey.pos][predator.pos] < min_utility:
                min_utility = utility[action][prey.pos][predator.pos]
                best_node = action
        
        agent_pos = best_node
        path.append(agent_pos)
        steps += 1
        
        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        #move predator
        predator.pred_distracted_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
    #fail if timeout error    
    if(steps >= 100):
        fail = 2 

    return fail, path

In [19]:
wins = 0
losses = 0
timeout = 0
times = 0

util = {}
for i in range(122316):
    x = np.expand_dims(total.iloc[i,:-1].values, axis = 1)
    y = network1.predict(x)
    util[x[0][0],x[1][0],x[2][0]] = y[0]

while times < 3000:
    
    res, path = AgentVStar(g, util)
    
    if(res == 0):
        wins += 1
    elif(res == 1):
        losses += 1
    else:
        timeout += 1
    times += 1
    print(times)
    
print('Number of wins: ' + str(wins))
print('Number of losses: ' + str(losses))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

KeyError: 1

In [20]:
class VStarPartial:
    
    def __init__(self,x,y):
        self.W0 = np.random.randn(53, 53)*0.01
        self.W1 = np.random.randn(106, 53)*0.01
        self.W2 = np.random.randn(1, 106)*0.01
        self.h0 = np.zeros((53, 1))
        self.h1 = np.zeros((106,1))
        self.h2 = np.zeros((1, 1))
        self.loss_array = []
        self.sigma0 = 0
        self.sigma1 = 0
        self.sigma2 = 0
        self.alpha0 = 0
        self.alpha1 = 0
        self.alpha2 = 0
        self.x = x.T
        self.y = y.T

    
    def forward(self):
        self.sigma0 = np.dot(self.W0,self.x) + self.h0
        self.sigma0 = np.random.choice([0, 1], size=(53,1), p=[0.3, 0.7])*self.sigma0 #sigma*dropout
        self.alpha0 = np.maximum(self.sigma0,0)
        
        self.sigma1 = np.dot(self.W1,self.alpha0) + self.h1
        self.sigma1 = np.random.choice([0, 1], size=(106,1), p=[0.3, 0.7])*self.sigma1 #sigma*dropout
        self.alpha1 = np.maximum(self.sigma1,0)
        
        self.sigma2 = np.dot(self.W2,self.alpha1) + self.h2
        self.alpha2 = self.sigma2
    
    def backward(self):
    
        back_sigma2 = (self.alpha2-self.y)
        back_W2 = (1/self.x.shape[0])*(np.dot(back_sigma2,self.alpha2.T)) + (0.7*self.W2)/(self.x.shape[0])
        back_h2 = (1/self.x.shape[0]) * np.sum(back_sigma2)

        back_sigma1 = np.multiply(np.dot(self.W2.T,back_sigma2), 1. * (self.sigma1 > 0))
        back_W1 = (1/self.x.shape[0])*(np.dot(back_sigma1,self.alpha1.T)) + (0.7*self.W1)/(self.x.shape[0])
        back_h1 = (1/self.x.shape[0]) * np.sum(back_sigma1)

        back_sigma0 = np.multiply(np.dot(self.W1.T,back_sigma1), 1. * (self.sigma0 > 0))
        back_W0 = (1/self.x.shape[0])*(np.dot(back_sigma0,self.x.T)) + (0.7*self.W0)/(self.x.shape[0])
        back_h0 = (1/self.x.shape[0])*(np.sum(back_sigma0))

        return back_W0, back_h0, back_W1, back_h1, back_W2, back_h2
    
    def run(self):
        for i in range(10000):
            self.forward()
            back_W0, back_h0, back_W1, back_h1, back_W2, back_h2 = self.backward()
            self.W0 = self.W0 - 0.001*back_W0
            self.h0 = self.h0 - 0.001*back_h0
            self.W1 = self.W1 - 0.001*back_W1
            self.h1 = self.h1 - 0.001*back_h1
            self.W2 = self.W2 - 0.001*back_W2
            self.h2 = self.h2 - 0.001*back_h2
            loss = (np.mean((self.y - self.alpha2)**2)*0.5)
            self.loss_array.append(loss)
            print("Loss at: ", i,loss)

In [21]:
#using a specific graph and UStar map to train the data
f = open('ustar-dion.p', 'rb')
utility = pickle.load(f)
f.close()
g = Graph()
g.graph = {1: [2, 50, 5], 2: [3, 1, 50], 3: [4, 2, 7], 4: [5, 3, 9], 5: [6, 4, 1], 6: [7, 5, 8], 7: [8, 6, 3], 8: [9, 7, 6], 9: [10, 8, 4], 10: [11, 9, 14], 11: [12, 10], 12: [13, 11, 16], 13: [14, 12, 18], 14: [15, 13, 10], 15: [16, 14, 19], 16: [17, 15, 12], 17: [18, 16, 22], 18: [19, 17, 13], 19: [20, 18, 15], 20: [21, 19, 24], 21: [22, 20, 26], 22: [23, 21, 17], 23: [24, 22, 27], 24: [25, 23, 20], 25: [26, 24, 29], 26: [27, 25, 21], 27: [28, 26, 23], 28: [29, 27, 30], 29: [30, 28, 25], 30: [31, 29, 28], 31: [32, 30, 34], 32: [33, 31], 33: [34, 32, 36], 34: [35, 33, 31], 35: [36, 34, 39], 36: [37, 35, 33], 37: [38, 36, 40], 38: [39, 37, 41], 39: [40, 38, 35], 40: [41, 39, 37], 41: [42, 40, 38], 42: [43, 41, 45], 43: [44, 42, 48], 44: [45, 43, 46], 45: [46, 44, 42], 46: [47, 45, 44], 47: [48, 46, 49], 48: [49, 47, 43], 49: [50, 48, 47], 50: [1, 49, 2]}
g.paths = g.all_shortest_paths()

In [ ]:
#saving all the positions, distances, and ustar values in a dataframe for each state for training and testing the neural network
util = []
for pos,val in utility.items():
    local = {}
    local["Agent"] = pos[0]
    local["Prey"] = pos[1]
    local["Predator"] = pos[2]
    local["Distance between Agent and Prey"] = len(g.paths.get((pos[0],pos[1]))[0])
    local["Distance between Agent and Predator"] = len(g.paths.get((pos[0],pos[2]))[0])
    local["Distance between Prey and Predator"] = len(g.paths.get((pos[2],pos[1]))[0])
    local["Ustar"] = val
    util.append(local)

#total = dataframe for training and testing
total = pd.DataFrame(util)
total_copy = total.copy()
total = total[total['Ustar'] < 100]

#x = agent, prey, and predator positions and distances between them
x = total.iloc[:,0:6].values
#y = UStar values
y = np.expand_dims(total.iloc[:,6].values, axis = 1)

#creating the training and testing data
train1, train2, test1, test2 = [], [], [], []

#70% of the total dataframe are for training, 30% are for testing
for i in range(len(y)):
    prob = np.random.randn()
    if prob < 0.7:
        train1.append(total.iloc[i,:-1].values)
        train2.append(total.iloc[i,-1])
    else:
        test1.append(total.iloc[i,:-1].values)
        test2.append(total.iloc[i,-1])

train1 = np.array(train1)
train2 = np.expand_dims(np.array(train2), axis = 1)
test1 = np.array(test1)
test2 = np.expand_dims(test2, axis = 1)


In [ ]:
network2 = VStarPartial(x,y)
network2.run()

In [ ]:
def AgentVStarPartial(g, utility):
    agent_pos = random.randint(1,50)
    predator = Predator(agent_pos)
    prey = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    
    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector[agent_pos-1] #obviously prey isnt at agents position so get probability of prey not being in that position
        belief_vector[agent_pos-1] = 0 #making belief of prey in agents node 0
        for i in range(len(belief_vector)):
                belief_vector[i] = belief_vector[i]/prey_not #normalizing
    
    while (fail == 0 and steps <= 100):

        max_prob = max(belief_vector)
        
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob) #randomly selecting one of those positions
        
        best_node = agent_pos
        min_utility = 10000
        
        for action in g.graph[agent_pos]:
            if belief_vector[action-1]*utility.Ustar[action][possible_prey_pos][predator.pos] < min_utility:
                #getting the U* partial value
                min_utility = belief_vector[action-1]*utility.Ustar[action][possible_prey_pos][predator.pos]
                best_node = action
        
        agent_pos = best_node
        path.append(agent_pos)
        steps += 1
        
        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break
        else: #if prey not caught, treat as survey and perform another belief update
            prey_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/prey_not

        #move predator and prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator.pred_distracted_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for next timestep
        temp_belief_vector = belief_vector.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector)):
            mag_sum =  temp_belief_vector[i]/(len(g.graph[i+1])+1) #probability of prey staying in same position
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            for bloke in i_neighbours:
                mag_sum += (temp_belief_vector[bloke-1]/(len(g.graph[bloke])+1)) #calculating probability of prey moving to node i if it was in node bloke
            belief_vector[i] = mag_sum #assigning probability
        
    #fail if timeout error    
    if(steps >= 100):
        fail = 2 

    return fail, path

In [ ]:
wins = 0
losses = 0
timeout = 0
times = 0

util = {}
for i in range(122316):
    x = np.expand_dims(total.iloc[i,:-1].values, axis = 1)
    y = network1.predict(x)
    util[[x[0][0],x[1][0],x[2][0]]] = y[0]

while times < 3000:
    
    res, path = AgentVStarPartial(g, util)
    
    if(res == 0):
        wins += 1
    elif(res == 1):
        losses += 1
    else:
        timeout += 1
    times += 1

print('Number of wins: ' + str(wins))
print('Number of losses: ' + str(losses))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))